# CUSTOMER DATA PROCESSING

## Solution 1

Reading in the provided ACW Data using the CSV library.

In [ ]:
import csv

#Path to the csv user file
csv_file_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/acw_user_data.csv" 


# This is just a test comment

#Opens a CSV file in read mode and assigns it to the fcsv file
with open (csv_file_path, "r") as csv_file:
    csv_reader = csv.reader(csv_file) #Creates a csv file reader object

## Solution 2

Converting the actual data from all flat structures into nested structures.

In [147]:
import csv
import json
 
json_file_path = csv_file_path.replace(".csv", "_nested.json")

with open(csv_file_path, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    result = []

    for row in csv_reader:
        
         person_data = {
             "first_name" : row["First Name"],
            "last_name" : row["Last Name"],
            "age" : int(row["Age (Years)"]),
             "sex" : row["Sex"],
            "marital_status" : row["Marital Status"],
            "dependants" : row["Dependants"],
            "employer_company" : row["Employer Company"],
            "yearly_pension" : float(row["Yearly Pension (Dollar)"]),
            "yearly_salary" : float(row["Yearly Salary (Dollar)"]),
            "retired" : row["Retired"],
            "Distance_commuted_km" : float(row["Distance Commuted to Work (Km)"]),

            "Address": {
                "Address Street" : row["Address Street"],
                "Address City" : row["Address City"],
                "Address Postcode" : row["Address Postcode"]
            },

            "Credit_Card": {
                "start_date" : row["Credit Card Start Date"],
                "expiry_date" : row["Credit Card Expiry Date"],
                "card_number" : row["Credit Card Number"],
                "security_number" : row["Credit Card CVV"],
                "iban" : row["Bank IBAN"],
            },

            "Vehicle": {
                 "make" : row["Vehicle Make"],
                 "model" : row["Vehicle Model"],
                 "year" : row["Vehicle Year"],
                 "type" : row["Vehicle Type"]
            }
         }

         result.append(person_data)

with open(json_file_path, "w") as json_file:
    json.dump(result, json_file, indent = 4)

## Solution 3

Due to the difficulty encountered, as a result of errors, of which the client has informed me as well, a new list will be created beleow to correct all of such error.

In [121]:
import csv

missing_dependants_rows = []
corrected_dependants_rows = []



with open (csv_file_path, newline="") as csv_file:
    csv_reader = list(csv.DictReader(csv_file))
    header_column = csv_reader[0].keys()

    for i, row in enumerate(csv_reader):
        dependants = row.get("Dependants", "").strip()
       
        if dependants == "":
            row["Dependants"] = "Not given"
            missing_dependants_rows.append(i + 2)
        corrected_dependants_rows.append(row)

print(f"missing_dependants_rows: {missing_dependants_rows}")

missing_dependants_rows: [23, 111, 181, 207, 272, 274, 276, 360, 462, 470, 581, 638, 681, 727, 824, 867, 919, 933, 985]


## Solution 4

A new record will be written into a processed.json file in the JSON data format.

In [36]:
import csv
import json

json_file_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/processed.json.json" 

with open(csv_file_path, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    result = []

    for row in csv_reader:
        
         person_data = {
             "first_name" : row["First Name"],
            "last_name" : row["Last Name"],
            "age" : int(row["Age (Years)"]),
             "sex" : row["Sex"],
            "marital_status" : row["Marital Status"],
            "dependants" : row["Dependants"],
            "employer_company" : row["Employer Company"],
            "yearly_pension" : float(row["Yearly Pension (Dollar)"]),
            "yearly_salary" : float(row["Yearly Salary (Dollar)"]),
            "retired" : row["Retired"],
            "Distance_commuted_km" : float(row["Distance Commuted to Work (Km)"]),

            "Address": {
                "Address Street" : row["Address Street"],
                "Address City" : row["Address City"],
                "Address Postcode" : row["Address Postcode"]
            },

            "Credit_Card": {
                "start_date" : row["Credit Card Start Date"],
                "expiry_date" : row["Credit Card Expiry Date"],
                "card_number" : row["Credit Card Number"],
                "security_number" : row["Credit Card CVV"],
                "iban" : row["Bank IBAN"],
            },

            "Vehicle": {
                 "make" : row["Vehicle Make"],
                 "model" : row["Vehicle Model"],
                 "year" : row["Vehicle Year"],
                 "type" : row["Vehicle Type"]
            }
         }

         result.append(person_data)

with open(json_file_path, "w") as json_file:
    json.dump(result, json_file, indent = 4)

## Solution 5

Two additional file outputs will be created for the retired customers(retired.json) and the employed customers(employed.json).

In [65]:
import csv
import json

# Ouput files
retired_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/retired.json.json" 
employed_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/employed.json.json" 

# Filtered output lists
retired_customers = []
employed_customers = []


with open(csv_file_path, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file)

    for row in csv_reader:
        
         person_data = {
             "first_name" : row["First Name"],
            "last_name" : row["Last Name"],
            "age" : int(row["Age (Years)"]),
             "sex" : row["Sex"],
            "marital_status" : row["Marital Status"],
            "dependants" : row["Dependants"],
            "employer_company" : row["Employer Company"],
            "yearly_pension" : float(row["Yearly Pension (Dollar)"]),
            "yearly_salary" : float(row["Yearly Salary (Dollar)"]),
            "retired" : row["Retired"],
            "Distance_commuted_km" : float(row["Distance Commuted to Work (Km)"]),

            "Address": {
                "Address Street" : row["Address Street"],
                "Address City" : row["Address City"],
                "Address Postcode" : row["Address Postcode"]
            },

            "Credit_Card": {
                "start_date" : row["Credit Card Start Date"],
                "expiry_date" : row["Credit Card Expiry Date"],
                "card_number" : row["Credit Card Number"],
                "security_number" : row["Credit Card CVV"],
                "iban" : row["Bank IBAN"],
            },

            "Vehicle": {
                 "make" : row["Vehicle Make"],
                 "model" : row["Vehicle Model"],
                 "year" : row["Vehicle Year"],
                 "type" : row["Vehicle Type"]
            }
         }
        
         if row["Retired"] == "True":
            retired_customers.append(person_data)

         if row["Employer Company"] != "N/A":
            employed_customers.append(person_data)



with open(retired_path, "w") as json_file:
    json.dump(retired_customers, json_file, indent = 4)

with open(employed_path, "w") as json_file:
    json.dump(employed_customers, json_file, indent = 4)

## Solution 6

Data of customers who have more than 10 years between their start and end date, seperated and stored on a remove_ccard.json file.

In [77]:
import csv
import json
from datetime import datetime

# Output files
remove_ccard_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/remove_ccard.json.json" 

flagged_ccards = []

def ccards_validity(row):
    start = row["Credit Card Start Date"]
    end = row["Credit Card Expiry Date"]

    start_month, start_year = start.split("/")
    end_month, end_year = end.split("/")

    start_month = int(start_month)
    end_month = int(end_month)
    start_year = int(start_year) + 2000
    end_year = int(end_year) + 2000


    total_months = (end_year - start_year) * 12 +(end_month - start_month)

    return total_months > 120

with open(csv_file_path, "r") as csv_file:
    csv_reader = csv.DictReader(csv_file)

    for row in csv_reader:
        if ccards_validity(row):
            flagged_ccards.append({
                "first_name" : row["First Name"],
                "last_name" : row["Last Name"],
                "start_date" : row["Credit Card Start Date"],
                "expiry_date" : row["Credit Card Expiry Date"],
                "card_number" : row["Credit Card Number"]
            })

with open(remove_ccard_path, "w") as json_file:
    json.dump(flagged_ccards, json_file, indent = 4)
              

## Solution 7

A new data attribute for customers "Salary-Commute" to calculate in additional metrics which would be used for ranking cusomers.

In [125]:
import json

# File path
json_file_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/processed.json.json"

with open (json_file_path, "r") as json_file:
    customer_data = json.load(json_file)


### 7a
Calculating  the new attribute (Salary-Commute), representing Salary customers earn, per Km of their commute.

In [124]:
for customer in customer_data:
    salary = customer["yearly_salary"]
    commute = customer["Distance_commuted_km"]

    if type(commute) == list and len(commute) > 0:
        commute = commute[0] # gets the first item on list

    if type(commute) == str:
        commute = commute.strip()
        if commute != "" and commute != "N/A":
            commute = 0
        else:
            commute = float(commute)
    if type(commute) != float and type(commute) != int:
        commute = 0
    if commute <= 1:
        customer ["Salary-Commute"] = salary
    else:
        customer ["Salary-Commute"] = round(salary / commute, 2)

with open(json_file_path, "w" ) as json_file:
    json.dump(data, json_file, indent = 4)

### 7b

Data is sorted in ascending order.

In [136]:
with open (json_file_path, "r") as json_file:
    customer_data = json.load(json_file)

def commute_value(customer):
    return customer["Salary-Commute"]

customer_data.sort(key=commute_value)

### 7c
The sorted output file will be stored as a JSON file "commute".

In [137]:
import json

# File path
json_file_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/processed.json.json"
commute_file_path = "C:/Users/USER/OneDrive - hull.ac.uk/Python/commute.json"

with open (json_file_path, "r") as json_file:
    customer_data = json.load(json_file)

def commute_value(customer):
    return customer["Salary-Commute"]

customer_data.sort(key=commute_value)


with open(commute_file_path, "w") as json_file:
    json.dump(customer_data, json_file, indent = 4)

#

#

# CUSTOMER DATA VISUALIZATION

Visualization method will be introduced here to make client understand the data they have on their customers a bit more. This will be done by the use of Pandas and seaborn, reading through the original CSV file

## Solution 1
Obtaining the data series for salary and Age and calculating the Mean and Median salary

In [3]:
import pandas as pd
import seaborn as sns
import csv

# Reads the file
df = pd.read_csv(csv_file_path)

### 1a
Mean Salary

In [4]:
salary = df["Yearly Salary (Dollar)"]

mean_salary = salary.mean()

mean_salary

np.float64(57814.078)

### 1b

Median Salary

In [6]:
median_salary = salary.median()

median_salary

np.float64(57518.0)

## Solution 2
A univariate plots will be performed on the following data attributes

### 2a
Age, calculating how many bins would be reqired for a bin_width of 5

In [12]:
age_column = df["Age (Years)"]

min_age = age_column.min()
max_age = age_column.max()
bin_width = 5

bin_num = (int(max_age - min_age) / bin_width)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (8,5))
sns.histplot(age_column)